Generating data for 10 hours

In [2]:
#!mkdir data

In [1]:
!python generate_ds.py --total_seconds 36000 --seed 26 --offset 0 --model_dir robot_reboot_model/model_0/

INFO:root:Running for 36000 seconds with seed 26
INFO:root:Working with 12 cores
INFO:root:Total time 0.0028221607208251953/36000
INFO:root:Total time 0.0044422149658203125/36000
INFO:root:Total time 0.0061261653900146484/36000
INFO:root:Total time 0.007838249206542969/36000
INFO:root:Total time 0.009722232818603516/36000
INFO:root:Total time 0.011712074279785156/36000
INFO:root:Total time 0.01452016830444336/36000
INFO:root:Total time 0.017274141311645508/36000
INFO:root:Total time 0.020135164260864258/36000
INFO:root:Total time 0.02305316925048828/36000
INFO:root:Total time 0.025780439376831055/36000
INFO:root:Total time 0.0294952392578125/36000
INFO:root:Processing jobs 0.0294952392578125/36000
2021-04-09 23:47:11.239280: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-09 23:47:11.240279: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library 

INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_0.tfrecords
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_4.tfrecords
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_3.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_11.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_6.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_7.tfrecords
INFO:root

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created

...

2021-04-10 09:04:20.774840: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:04:20.775889: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:04:20.785457: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:04:20.786779: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:04:20.786885: I tensorflow/com

2021-04-10 09:17:35.782211: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:17:35.783547: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:17:35.784576: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:17:35.786969: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:17:35.788078: I tensorflow/cor

2021-04-10 09:30:17.949745: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:30:17.952038: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:30:17.953214: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-10 09:30:17.955936: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 09:30:17.957131: I tensorflow/cor

INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_607.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_604.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_609.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_608.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_601.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:ro

2021-04-10 09:40:32.442905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.446763: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.448585: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.450288: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.450627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.450710: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 09:40:32.451999: I tensorflow/compiler/mlir/mlir_grap

In [3]:
!python generate_ds.py --total_seconds 36000 --seed 18 --offset 624 --model_dir robot_reboot_model/model_0/

INFO:root:Running for 36000 seconds with seed 18
INFO:root:Working with 12 cores
INFO:root:Total time 0.0048329830169677734/36000
INFO:root:Total time 0.006237030029296875/36000
INFO:root:Total time 0.007879972457885742/36000
INFO:root:Total time 0.009547948837280273/36000
INFO:root:Total time 0.011243104934692383/36000
INFO:root:Total time 0.012993812561035156/36000
INFO:root:Total time 0.014941930770874023/36000
INFO:root:Total time 0.017121076583862305/36000
INFO:root:Total time 0.019884824752807617/36000
INFO:root:Total time 0.02271580696105957/36000
INFO:root:Total time 0.025918006896972656/36000
INFO:root:Total time 0.029448986053466797/36000
INFO:root:Processing jobs 0.029448986053466797/36000
2021-04-10 23:46:40.164400: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-10 23:46:40.165563: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Libra

2021-04-10 23:46:40.525885: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-04-10 23:46:40.563625: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_633.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_627.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_631.tfrecords
INFO:root:Search finished, probabilities calculated
INFO:root:Playing the game
INFO:root:Game outcome calculated
INFO:root:Saving file data/robot_reboot_632.tfrecords
INFO:root:Search fin

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created

2021-04-11 00:08:35.754596: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-11 00:08:35.759454: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-11 00:08:35.760634: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search 

INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created

...

2021-04-11 08:59:31.488138: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-11 08:59:31.489047: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-11 08:59:31.489257: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-11 08:59:31.490296: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-11 08:59:31.497829: I tensorflow/com

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

2021-04-11 09:33:45.668661: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-11 09:33:45.671622: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-11 09:33:45.672966: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-11 09:33:45.673119: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-04-11 09:33:45.674038: I tensorflow/cor

INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Starting to create a dataset
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Starting to create a dataset
INFO:root:Starting to create a dataset
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:Game created
INFO:root:Search using MCTS
INFO:root:

In [ ]:
!python generate_ds.py --total_samples 1000 --seed 28 --offset 1297 --model_dir robot_reboot_model/model_0/

In [ ]:
#!python generate_ds.py --total_samples 500 --seed 4 --offset 3000 --model_dir robot_reboot_model/model_0/

In [2]:
#!python generate_ds.py --total_samples 500 --seed 2601 --offset 3500 --model_dir robot_reboot_model/model_0/

In [1]:
!python3 generate_validation_ds.py --total_samples 400 --seed 260108

INFO:root:Generating 400 samples with seed 260108
INFO:root:Finished after 0.8786880970001221 seconds
